# 12.4_Class_Decorators

In [1]:
def decorate(cls):
    print('Decorating', cls)
    return cls

In [2]:
@decorate
class Spam:
    def yow(self):
        print('Yow')

Decorating <class '__main__.Spam'>


## add logging for all the methods in Spam class
- doing brain surgery on methods

In [6]:
class Foo:
    def bar(self):
        pass

In [7]:
Foo

__main__.Foo

In [8]:
vars(Foo)

mappingproxy({'__module__': '__main__',
              'bar': <function __main__.Foo.bar(self)>,
              '__dict__': <attribute '__dict__' of 'Foo' objects>,
              '__weakref__': <attribute '__weakref__' of 'Foo' objects>,
              '__doc__': None})

In [9]:
vars(Foo).items()

dict_items([('__module__', '__main__'), ('bar', <function Foo.bar at 0x7fc3944c8200>), ('__dict__', <attribute '__dict__' of 'Foo' objects>), ('__weakref__', <attribute '__weakref__' of 'Foo' objects>), ('__doc__', None)])

In [10]:
list(vars(Foo).items())

[('__module__', '__main__'),
 ('bar', <function __main__.Foo.bar(self)>),
 ('__dict__', <attribute '__dict__' of 'Foo' objects>),
 ('__weakref__', <attribute '__weakref__' of 'Foo' objects>),
 ('__doc__', None)]

In [11]:
def add():
    pass

In [12]:
callable(add)

True

In [1]:
from functools import wraps

def logged(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        print(f'Calling {func.__name__}')
        return func(*args, **kwargs)
    return wrapper

In [2]:
def logmethods(cls):
    for key, value in list(vars(cls).items()):
        if callable(value):
            setattr(cls, key, logged(value))
    return cls

In [3]:
@logmethods
class Spam:
    def __init__(self, value):
        self.value = value
    
    def yow(self):
        print('Yow!')
        
    def grok(self):
        print('Grok!')

In [4]:
s = Spam(2)

Calling __init__


In [5]:
s.yow()

Calling yow
Yow!


In [6]:
s.grok()

Calling grok
Grok!


## add validation for class attributes

In [9]:
class Typed:
    expected_type = object
    
    def __init__(self, name):
        self.name = name
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError(f'Expected {self.expected_type}')
            
class Integer(Typed):
    expected_type = int
    
class Marks:
    value = Integer('value')
    
    def __init__(self, value):
        self.value = value

In [10]:
m = Marks(23)

In [11]:
m.value = 43

In [12]:
m.value = 'hi'

TypeError: Expected <class 'int'>

### modify using class decorators
- change name in Typed `__init__` to be None

In [14]:
class Typed:
    expected_type = object
    
    def __init__(self, name=None):
        self.name = name
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError(f'Expected {self.expected_type}')
            
class Integer(Typed):
    expected_type = int
    
def typed(cls):
    for key, value in vars(cls).items():
        if isinstance(value, Typed):
            value.name = key
    return cls
    
@typed
class Marks:
    value = Integer()
    
    def __init__(self, value):
        self.value = value

In [15]:
m = Marks(23)

In [16]:
m.value = 43

In [17]:
m.value = 'hi'

TypeError: Expected <class 'int'>

## use decorator with arguments

In [19]:
class Typed:
    expected_type = object
    
    def __init__(self, name=None):
        self.name = name
        
    def __get__(self, instance, cls):
        return instance.__dict__[self.name]
    
    def __set__(self, instance, value):
        if not isinstance(value, self.expected_type):
            raise TypeError(f'Expected {self.expected_type}')
            
class Integer(Typed):
    expected_type = int
    
def validate(**kwargs):
    def decorate(cls):
        for name, val in kwargs.items():
            setattr(cls, name, val(name))
        return cls
    return decorate
    
@validate(value=Integer)
class Marks:
    def __init__(self, value):
        self.value = value

In [20]:
m = Marks(23)

In [21]:
m.value = 43

In [22]:
m.value = 'hi'

TypeError: Expected <class 'int'>